# Tune hyperparameters with the Keras Tuner

https://www.tensorflow.org/tutorials/keras/keras_tuner

## Import Keras

In [1]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

In [3]:
# Setup memory to fix critical issue
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [24]:
# test running device
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Num GPUs Available:  1
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [27]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


## Dataset Example

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model

In [17]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

### Instantiate the tuner and perform hypertuning

In [18]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [19]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [20]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 30s]
val_accuracy: 0.8484166860580444

Best val_accuracy So Far: 0.8915833234786987
Total elapsed time: 00h 07m 22s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 224 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

In [21]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# use cross validation accuracy maximum's epoch as the best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6217 - accuracy: 0.7792 - val_loss: 0.4288 - val_accuracy: 0.8437
Epoch 2/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3718 - accuracy: 0.8629 - val_loss: 0.3659 - val_accuracy: 0.8668
Epoch 3/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3305 - accuracy: 0.8767 - val_loss: 0.3816 - val_accuracy: 0.8618
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2949 - accuracy: 0.8893 - val_loss: 0.3584 - val_accuracy: 0.8648
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2837 - accuracy: 0.8942 - val_loss: 0.3311 - val_accuracy: 0.8823
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2693 - accuracy: 0.9012 - val_loss: 0.3256 - val_accuracy: 0.8861
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2539 - accuracy: 0.9031 - val_loss: 0.3074 - val_accuracy:

In [22]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_test, label_test, epochs=best_epoch)

Epoch 1/28
313/313 [==============================] - 1s 2ms/step - loss: 0.9063 - accuracy: 0.6794
Epoch 2/28
313/313 [==============================] - 1s 2ms/step - loss: 0.5069 - accuracy: 0.8153
Epoch 3/28
313/313 [==============================] - 1s 2ms/step - loss: 0.4440 - accuracy: 0.8415
Epoch 4/28
313/313 [==============================] - 1s 2ms/step - loss: 0.3917 - accuracy: 0.8603
Epoch 5/28
313/313 [==============================] - 1s 2ms/step - loss: 0.3658 - accuracy: 0.8653
Epoch 6/28
313/313 [==============================] - 1s 2ms/step - loss: 0.3398 - accuracy: 0.8699
Epoch 7/28
313/313 [==============================] - 1s 2ms/step - loss: 0.3287 - accuracy: 0.8721
Epoch 8/28
313/313 [==============================] - 1s 2ms/step - loss: 0.3118 - accuracy: 0.8841
Epoch 9/28
313/313 [==============================] - 1s 2ms/step - loss: 0.2978 - accuracy: 0.8870
Epoch 10/28
313/313 [==============================] - 1s 2ms/step - loss: 0.2817 - accuracy: 0.8974

In [23]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.1169 - accuracy: 0.9579
[test loss, test accuracy]: [0.11690054833889008, 0.9578999876976013]
